In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import json

## Defining Business Problem



In [ ]:
# Open the folder of csv files and concatenate them into one dataframe
# TODO save concatenated dataframe as csv file
path = r'./data/data-2'
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f) for f in all_files)
pd.concat(df_from_each_file, ignore_index=True).to_csv('./data/merged.csv')

In [ ]:
df['category'].map(json.loads).apply(pd.Series).add_prefix('category.')

In [ ]:
# load json data from colums using json.loads
categories = df['category'].apply(json.loads)

df['category'] = [x['name'] for x in categories]


In [ ]:
# load json data from colums using json.loads
location = df['location'].apply(json.dumps)

location = df['location'].apply(json.loads)

location

In [ ]:
df = pd.read_csv('./data/merged.csv')
df.head()

In [ ]:
# TODO Drop undesired categories
# ? How to define success or failure
df['state'].value_counts()
# get percentage of successful projects
df['state'].value_counts(normalize=True)


In [ ]:
features = ['category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged', 'usd_pledged_real', 'usd_goal_real']

In [ ]:
df.info()

In [ ]:
df.iloc[:,0:15].head()

In [ ]:
df.iloc[:,15:25].head()

In [ ]:
df.iloc[:,25:36].head()

In [ ]:
df['location'].map(json.loads).apply(pd.Series).add_prefix('location.')